# Transfer learning, finetune VGG16

## 1. Why VGG16

VGG16 is a convolutional neural network model proposed by K. Simonyan and A. Zisserman from the University of Oxford in the paper "Very Deep Convolutional Networks for Large-Scale Image Recognition". It is one of the most popular pre-trained models for image classification tasks.

### Key Features of VGG16
1. Architecture: VGG16 is a convolutional neural network model proposed by K. Simonyan and A. Zisserman from the University of Oxford in the paper "Very Deep Convolutional Networks for Large-Scale Image Recognition".<br>
2. Layers: It has 16 layers with learnable weights, including 13 convolutional layers and 3 fully connected layers.<br>
3. Pre-trained Weights: The model is pre-trained on the ImageNet dataset, which contains 1.2 million images and 1000 classes.<br>
4. Usage: It is widely used for image classification tasks and can be fine-tuned for specific tasks such as brain tumor classification.

## 2. Finetune VGG16 for brain tumor classification task

### 2.1 Preprocessing data

In [1]:
import cv2
import os
import random
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from keras.utils import normalize
from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
seed = 99
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

#### Reading and nomalize dataset

In [3]:
no_dir = os.listdir('./data_no/data_no/NO/')
yes_dir = os.listdir('./data_yes/data_yes/YES/')
data_set,label = [],[]
for i,cur_img_dir in enumerate(no_dir):
    #check type of image
    if cur_img_dir.split('.')[1]=='jpg':
        img = cv2.imread('./data_no/data_no/NO/'+cur_img_dir)
        img = Image.fromarray(img,'RGB')
        img = img.resize((64,64))
        data_set.append(np.array(img))
        label.append(0)
for i,cur_img_dir in enumerate(yes_dir):
    #check type of image
    if cur_img_dir.split('.')[1]=='jpg':
        img = cv2.imread('./data_yes/data_yes/YES/'+cur_img_dir)
        img = Image.fromarray(img,'RGB')
        img = img.resize((64,64))
        data_set.append(np.array(img))
        label.append(1)

In [4]:
data_set = np.array(data_set)
label = np.array(label)

#### Splitting the data into training and testing

In [5]:
x_train,x_test,y_train,y_test = train_test_split(
    data_set,label,
    test_size=0.2,
    random_state=99
    )
x_train,x_val,y_train,y_val = train_test_split(
        x_train,y_train,
    test_size=0.25,
    random_state=9
)

In [6]:
print(f'X train shape: {x_train.shape}\nY train shape: {y_train.shape}\nX test shape: {x_test.shape}\nY test shape: {y_test.shape})')

X train shape: (654, 64, 64, 3)
Y train shape: (654,)
X test shape: (219, 64, 64, 3)
Y test shape: (219,))


#### Nomalize

*update 25/08 using better nomalize method*

In [9]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import joblib 
# using scaler of nb4
scaler = joblib.load('scaler.pkl') 
# Reshape data to fit with MinMaxScaler
x_train_reshaped = x_train.reshape(-1, x_train.shape[-1])
x_test_reshaped = x_test.reshape(-1, x_test.shape[-1])
x_val_reshaped = x_val.reshape(-1, x_val.shape[-1])

x_train_reshaped = scaler.fit_transform(x_train_reshaped)
x_test_reshaped = scaler.transform(x_test_reshaped)
x_val_reshaped = scaler.transform(x_val_reshaped)

# Reshape to original shape
x_train = x_train_reshaped.reshape(x_train.shape)
x_test = x_test_reshaped.reshape(x_test.shape)
x_val = x_val_reshaped.reshape(x_val.shape)

Save new scaler method to using with new image in the future

In [10]:
import joblib

# Store scaler element
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

### Loading pre-trained model

In [11]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
# Load the VGG16 model with pre-trained weights, excluding the top fully connected layers
base_model = VGG16(
    weights='imagenet', # bo hoac none
    include_top=False, 
    input_shape=(64, 64, 3)
    )

# Freeze the convolutional base
for layer in base_model.layers:
    layer.trainable = False # True for fine tune model


In [13]:
# Add custom top layers for classification
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dense(1, activation='sigmoid')(x)  #binary classification 
# note 1 dense +1 activation

In [14]:
# Create the new model
model = Model(inputs=base_model.input, outputs=x)

In [15]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 64, 64, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 32, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 16, 16, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 16, 16, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 16, 16, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 8, 8, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 8, 8, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 4, 4, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 2, 2, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,049,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           513 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,764,289 (60.14 MB)

 Trainable params: 1,049,601 (4.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

*update 2508- adding early stopping method*

In [19]:
model.compile(
    optimizer=Adam(learning_rate=1e-4, amsgrad=True),
    loss='binary_crossentropy', 
    metrics=['accuracy']
    )
# Define EarlyStopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=3,          # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)
# Train the model
history = model.fit(
    x_train, 
    y_train, 
    epochs=50, 
    batch_size=32, 
    shuffle=False,
    validation_data = (x_test, y_test),
    callbacks=[early_stopping]  # Add EarlyStopping callback
    )

Epoch 1/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 17s 747ms/step - accuracy: 0.6275 - loss: 0.6345 - val_accuracy: 0.7991 - val_loss: 0.5172
Epoch 2/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 13s 613ms/step - accuracy: 0.7442 - loss: 0.5241 - val_accuracy: 0.7945 - val_loss: 0.4714
Epoch 3/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 661ms/step - accuracy: 0.7556 - loss: 0.4975 - val_accuracy: 0.8037 - val_loss: 0.4520
Epoch 4/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 645ms/step - accuracy: 0.7606 - loss: 0.4812 - val_accuracy: 0.8082 - val_loss: 0.4375
Epoch 5/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 664ms/step - accuracy: 0.7699 - loss: 0.4671 - val_accuracy: 0.8128 - val_loss: 0.4246
Epoch 6/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 13s 628ms/step - accuracy: 0.7789 - loss: 0.4544 - val_accuracy: 0.8265 - val_loss: 0.4128
Epoch 7/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 686ms/step - accuracy: 0.7907 - loss: 0.4426 - val_accuracy: 0.8447 - val_loss: 0.4025
Epoch 8/50
21/21 ━━━━━━━━━━━━━━━━━━━━ 14s 682ms/step - accuracy: 0.7963 - loss: 0.4315 - val_accu

Test with new data

In [21]:
y_test_pred = model.predict(x_test)
y_pred = (y_test_pred >0.5).astype(int)
y_test_reshape = y_test.reshape(-1,1)
print("Accuracy in test set:", accuracy_score(y_test_reshape, y_pred))
print('Accuracy in validation set:',history.history['val_accuracy'][-1])

7/7 ━━━━━━━━━━━━━━━━━━━━ 5s 604ms/step
Accuracy in test set: 0.9269406392694064
Accuracy in validation set: 0.9269406199455261


**Infor**<br>
Name: BrainTurmor_v3<br>
Accuracy in test set: 0.9269406392694064<br>
Status: Good

In [22]:
model.save('BrainTurmor_v3.keras')